In [97]:
!nvidia-smi

Fri Aug 26 14:15:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:17:00.0 Off |                  N/A |
|  0%   35C    P8    16W / 350W |   2063MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:65:00.0 Off |                  N/A |
| 30%   

In [98]:
pwd

'/home/user/3min/Graph_rec_wwww19'

In [99]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import pickle
import numpy as np
import time
import random
from collections import defaultdict
#from UV_Encoders import UV_Encoder
#from UV_Aggregators import UV_Aggregator
#from Social_Encoders import Social_Encoder
#from Social_Aggregators import Social_Aggregator
import torch.nn.functional as F
import torch.utils.data
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import datetime
import argparse
import os



"""
## toy dataset 
history_u_lists, history_ur_lists:  user's purchased history (item set in training set), and his/her rating score (dict)
history_v_lists, history_vr_lists:  user set (in training set) who have interacted with the item, and rating score (dict)

train_u, train_v, train_r: training_set (user, item, rating)
test_u, test_v, test_r: testing set (user, item, rating)

# please add the validation set

social_adj_lists: user's connected neighborhoods
ratings_list: rating value from 0.5 to 4.0 (8 opinion embeddings)
"""

In [100]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

embed_dim = 128
dir_data = './data/toy_dataset'

path_data = dir_data + ".pickle"
data_file = open(path_data, 'rb')
history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, test_u, test_v, test_r, social_adj_lists, ratings_list = pickle.load(
    data_file)

In [101]:
trainset = torch.utils.data.TensorDataset(torch.LongTensor(train_u), torch.LongTensor(train_v),
                                              torch.FloatTensor(train_r))
testset = torch.utils.data.TensorDataset(torch.LongTensor(test_u), torch.LongTensor(test_v),
                                         torch.FloatTensor(test_r))

In [102]:
batch_size = 128
test_batch_size = 128

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=True)

# 705명 유저
num_users = history_u_lists.__len__()

# 1941개 아이템 수 
num_items = history_v_lists.__len__()

# 8등급, 2.0: 0, 1.0: 1, 3.0: 2, 4.0: 3, 2.5: 4, 3.5: 5, 1.5: 6, 0.5: 7
num_ratings = ratings_list.__len__()

In [103]:
ratings_list

{2.0: 0, 1.0: 1, 3.0: 2, 4.0: 3, 2.5: 4, 3.5: 5, 1.5: 6, 0.5: 7}

In [104]:
# embed_dim = 128차원 

u2e = nn.Embedding(num_users, embed_dim).to(device)
v2e = nn.Embedding(num_items, embed_dim).to(device)
r2e = nn.Embedding(num_ratings, embed_dim).to(device)

In [105]:
u2e, v2e, r2e

(Embedding(705, 128), Embedding(1941, 128), Embedding(8, 128))

In [106]:
import torch
import torch.nn as nn
from torch.nn import init
import numpy as np
import random
import torch.nn.functional as F


class Attention(nn.Module):
    def __init__(self, embedding_dims):
        super(Attention, self).__init__()
        self.embed_dim = embedding_dims
        self.bilinear = nn.Bilinear(self.embed_dim, self.embed_dim, 1)
        self.att1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.att2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.att3 = nn.Linear(self.embed_dim, 1)
        self.softmax = nn.Softmax(0)

    def forward(self, node1, u_rep, num_neighs):
        uv_reps = u_rep.repeat(num_neighs, 1)
        x = torch.cat((node1, uv_reps), 1)
        x = F.relu(self.att1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.att2(x))
        x = F.dropout(x, training=self.training)
        x = self.att3(x)
        att = F.softmax(x, dim=0)
        return att

In [107]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import random
#from Attention import Attention


class UV_Aggregator(nn.Module):
    """
    item and user aggregator: for aggregating embeddings of neighbors (item/user aggreagator).
    """

    def __init__(self, v2e, r2e, u2e, embed_dim, cuda="cpu", uv=True):
        super(UV_Aggregator, self).__init__()
        self.uv = uv
        self.v2e = v2e
        self.r2e = r2e
        self.u2e = u2e
        self.device = cuda
        self.embed_dim = embed_dim
        self.w_r1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.w_r2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.att = Attention(self.embed_dim)

    def forward(self, nodes, history_uv, history_r):

        embed_matrix = torch.empty(len(history_uv), self.embed_dim, dtype=torch.float).to(self.device)

        for i in range(len(history_uv)): 
            history = history_uv[i]
            num_histroy_item = len(history)
            tmp_label = history_r[i]

            if self.uv == True:
                # user component
                e_uv = self.v2e.weight[history]
                uv_rep = self.u2e.weight[nodes[i]]
            else:
                # item component
                e_uv = self.u2e.weight[history]
                uv_rep = self.v2e.weight[nodes[i]]

            e_r = self.r2e.weight[tmp_label]
            x = torch.cat((e_uv, e_r), 1)
            x = F.relu(self.w_r1(x))
            o_history = F.relu(self.w_r2(x))

            att_w = self.att(o_history, uv_rep, num_histroy_item)
            att_history = torch.mm(o_history.t(), att_w)
            att_history = att_history.t()

            embed_matrix[i] = att_history
        to_feats = embed_matrix
        return to_feats

In [108]:
agg_u_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=True) # user representation 

In [109]:
agg_u_history

UV_Aggregator(
  (v2e): Embedding(1941, 128)
  (r2e): Embedding(8, 128)
  (u2e): Embedding(705, 128)
  (w_r1): Linear(in_features=256, out_features=128, bias=True)
  (w_r2): Linear(in_features=128, out_features=128, bias=True)
  (att): Attention(
    (bilinear): Bilinear(in1_features=128, in2_features=128, out_features=1, bias=True)
    (att1): Linear(in_features=256, out_features=128, bias=True)
    (att2): Linear(in_features=128, out_features=128, bias=True)
    (att3): Linear(in_features=128, out_features=1, bias=True)
    (softmax): Softmax(dim=0)
  )
)

In [110]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F


class UV_Encoder(nn.Module):

    def __init__(self, features, embed_dim, history_uv_lists, history_r_lists, aggregator, cuda="cpu", uv=True):
        super(UV_Encoder, self).__init__()

        self.features = features
        self.uv = uv
        self.history_uv_lists = history_uv_lists
        self.history_r_lists = history_r_lists
        self.aggregator = aggregator
        self.embed_dim = embed_dim
        self.device = cuda
        self.linear1 = nn.Linear(2 * self.embed_dim, self.embed_dim)  #

    def forward(self, nodes):
        tmp_history_uv = []
        tmp_history_r = []
        for node in nodes:
            tmp_history_uv.append(self.history_uv_lists[int(node)])
            tmp_history_r.append(self.history_r_lists[int(node)])

        neigh_feats = self.aggregator.forward(nodes, tmp_history_uv, tmp_history_r)  # user-item network

        self_feats = self.features.weight[nodes]
        # self-connection could be considered.
        combined = torch.cat([self_feats, neigh_feats], dim=1)
        combined = F.relu(self.linear1(combined))

        return combined

In [111]:
enc_u_history = UV_Encoder(u2e, embed_dim, history_u_lists, history_ur_lists, agg_u_history, cuda=device, uv=True)

In [112]:
enc_u_history

UV_Encoder(
  (features): Embedding(705, 128)
  (aggregator): UV_Aggregator(
    (v2e): Embedding(1941, 128)
    (r2e): Embedding(8, 128)
    (u2e): Embedding(705, 128)
    (w_r1): Linear(in_features=256, out_features=128, bias=True)
    (w_r2): Linear(in_features=128, out_features=128, bias=True)
    (att): Attention(
      (bilinear): Bilinear(in1_features=128, in2_features=128, out_features=1, bias=True)
      (att1): Linear(in_features=256, out_features=128, bias=True)
      (att2): Linear(in_features=128, out_features=128, bias=True)
      (att3): Linear(in_features=128, out_features=1, bias=True)
      (softmax): Softmax(dim=0)
    )
  )
  (linear1): Linear(in_features=256, out_features=128, bias=True)
)

In [113]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import random
#from Attention import Attention


class Social_Aggregator(nn.Module):
    """
    Social Aggregator: for aggregating embeddings of social neighbors.
    """

    def __init__(self, features, u2e, embed_dim, cuda="cpu"):
        super(Social_Aggregator, self).__init__()

        self.features = features
        self.device = cuda
        self.u2e = u2e
        self.embed_dim = embed_dim
        self.att = Attention(self.embed_dim)

    def forward(self, nodes, to_neighs):
        embed_matrix = torch.empty(len(nodes), self.embed_dim, dtype=torch.float).to(self.device)
        for i in range(len(nodes)):
            tmp_adj = to_neighs[i]
            num_neighs = len(tmp_adj)
            # 
            e_u = self.u2e.weight[list(tmp_adj)] # fast: user embedding 
            #slow: item-space user latent factor (item aggregation)
            #feature_neigbhors = self.features(torch.LongTensor(list(tmp_adj)).to(self.device))
            #e_u = torch.t(feature_neigbhors)

            u_rep = self.u2e.weight[nodes[i]]

            att_w = self.att(e_u, u_rep, num_neighs)
            att_history = torch.mm(e_u.t(), att_w).t()
            embed_matrix[i] = att_history
        to_feats = embed_matrix

        return to_feats

In [114]:
lambda nodes: enc_u_history(nodes).t()

<function __main__.<lambda>(nodes)>

In [115]:
agg_u_social = Social_Aggregator(lambda nodes: enc_u_history(nodes).t(), u2e, embed_dim, cuda=device)

In [116]:
agg_u_social

Social_Aggregator(
  (u2e): Embedding(705, 128)
  (att): Attention(
    (bilinear): Bilinear(in1_features=128, in2_features=128, out_features=1, bias=True)
    (att1): Linear(in_features=256, out_features=128, bias=True)
    (att2): Linear(in_features=128, out_features=128, bias=True)
    (att3): Linear(in_features=128, out_features=1, bias=True)
    (softmax): Softmax(dim=0)
  )
)

In [117]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F


class Social_Encoder(nn.Module):

    def __init__(self, features, embed_dim, social_adj_lists, aggregator, base_model=None, cuda="cpu"):
        super(Social_Encoder, self).__init__()

        self.features = features
        self.social_adj_lists = social_adj_lists
        self.aggregator = aggregator
        if base_model != None:
            self.base_model = base_model
        self.embed_dim = embed_dim
        self.device = cuda
        self.linear1 = nn.Linear(2 * self.embed_dim, self.embed_dim)  #

    def forward(self, nodes):

        to_neighs = []
        for node in nodes:
            to_neighs.append(self.social_adj_lists[int(node)])
        neigh_feats = self.aggregator.forward(nodes, to_neighs)  # user-user network
        #print(type(self_feats))
        self_feats = self.features(torch.LongTensor(nodes.cpu().numpy())).to(self.device)
        #print(type(self_feats))
        self_feats = self_feats.t()
        
        # self-connection could be considered.
        combined = torch.cat([self_feats, neigh_feats], dim=1)
        combined = F.relu(self.linear1(combined))

        return combined

In [118]:
enc_u = Social_Encoder(lambda nodes: enc_u_history(nodes).t(), embed_dim, social_adj_lists, agg_u_social, base_model=enc_u_history, cuda=device)

In [119]:
enc_u

Social_Encoder(
  (aggregator): Social_Aggregator(
    (u2e): Embedding(705, 128)
    (att): Attention(
      (bilinear): Bilinear(in1_features=128, in2_features=128, out_features=1, bias=True)
      (att1): Linear(in_features=256, out_features=128, bias=True)
      (att2): Linear(in_features=128, out_features=128, bias=True)
      (att3): Linear(in_features=128, out_features=1, bias=True)
      (softmax): Softmax(dim=0)
    )
  )
  (base_model): UV_Encoder(
    (features): Embedding(705, 128)
    (aggregator): UV_Aggregator(
      (v2e): Embedding(1941, 128)
      (r2e): Embedding(8, 128)
      (u2e): Embedding(705, 128)
      (w_r1): Linear(in_features=256, out_features=128, bias=True)
      (w_r2): Linear(in_features=128, out_features=128, bias=True)
      (att): Attention(
        (bilinear): Bilinear(in1_features=128, in2_features=128, out_features=1, bias=True)
        (att1): Linear(in_features=256, out_features=128, bias=True)
        (att2): Linear(in_features=128, out_feature

In [120]:
# # item feature: user * rating
agg_v_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=False)
enc_v_history = UV_Encoder(v2e, embed_dim, history_v_lists, history_vr_lists, agg_v_history, cuda=device, uv=False)

In [121]:
agg_v_history

UV_Aggregator(
  (v2e): Embedding(1941, 128)
  (r2e): Embedding(8, 128)
  (u2e): Embedding(705, 128)
  (w_r1): Linear(in_features=256, out_features=128, bias=True)
  (w_r2): Linear(in_features=128, out_features=128, bias=True)
  (att): Attention(
    (bilinear): Bilinear(in1_features=128, in2_features=128, out_features=1, bias=True)
    (att1): Linear(in_features=256, out_features=128, bias=True)
    (att2): Linear(in_features=128, out_features=128, bias=True)
    (att3): Linear(in_features=128, out_features=1, bias=True)
    (softmax): Softmax(dim=0)
  )
)

In [122]:
enc_v_history

UV_Encoder(
  (features): Embedding(1941, 128)
  (aggregator): UV_Aggregator(
    (v2e): Embedding(1941, 128)
    (r2e): Embedding(8, 128)
    (u2e): Embedding(705, 128)
    (w_r1): Linear(in_features=256, out_features=128, bias=True)
    (w_r2): Linear(in_features=128, out_features=128, bias=True)
    (att): Attention(
      (bilinear): Bilinear(in1_features=128, in2_features=128, out_features=1, bias=True)
      (att1): Linear(in_features=256, out_features=128, bias=True)
      (att2): Linear(in_features=128, out_features=128, bias=True)
      (att3): Linear(in_features=128, out_features=1, bias=True)
      (softmax): Softmax(dim=0)
    )
  )
  (linear1): Linear(in_features=256, out_features=128, bias=True)
)

In [123]:
class GraphRec(nn.Module):

    def __init__(self, enc_u, enc_v_history, r2e):
        super(GraphRec, self).__init__()
        self.enc_u = enc_u
        self.enc_v_history = enc_v_history
        self.embed_dim = enc_u.embed_dim

        self.w_ur1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_ur2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_uv1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.w_uv2 = nn.Linear(self.embed_dim, 16)
        self.w_uv3 = nn.Linear(16, 1)
        self.r2e = r2e
        self.bn1 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn2 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn3 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn4 = nn.BatchNorm1d(16, momentum=0.5)
        self.criterion = nn.MSELoss()

    def forward(self, nodes_u, nodes_v):
        embeds_u = self.enc_u(nodes_u)
        embeds_v = self.enc_v_history(nodes_v)

        x_u = F.relu(self.bn1(self.w_ur1(embeds_u)))
        x_u = F.dropout(x_u, training=self.training)
        x_u = self.w_ur2(x_u)
        x_v = F.relu(self.bn2(self.w_vr1(embeds_v)))
        x_v = F.dropout(x_v, training=self.training)
        x_v = self.w_vr2(x_v)

        x_uv = torch.cat((x_u, x_v), 1)
        x = F.relu(self.bn3(self.w_uv1(x_uv)))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.bn4(self.w_uv2(x)))
        x = F.dropout(x, training=self.training)
        scores = self.w_uv3(x)
        return scores.squeeze()

    def loss(self, nodes_u, nodes_v, labels_list):
        scores = self.forward(nodes_u, nodes_v)
        return self.criterion(scores, labels_list)


def train(model, device, train_loader, optimizer, epoch, best_rmse, best_mae):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        batch_nodes_u, batch_nodes_v, labels_list = data
        optimizer.zero_grad()
        loss = model.loss(batch_nodes_u.to(device), batch_nodes_v.to(device), labels_list.to(device))
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f, The best rmse/mae: %.6f / %.6f' % (
                epoch, i, running_loss / 100, best_rmse, best_mae))
            running_loss = 0.0
    return 0


def test(model, device, test_loader):
    model.eval()
    tmp_pred = []
    target = []
    with torch.no_grad():
        for test_u, test_v, tmp_target in test_loader:
            test_u, test_v, tmp_target = test_u.to(device), test_v.to(device), tmp_target.to(device)
            val_output = model.forward(test_u, test_v)
            tmp_pred.append(list(val_output.data.cpu().numpy()))
            target.append(list(tmp_target.data.cpu().numpy()))
    tmp_pred = np.array(sum(tmp_pred, []))
    target = np.array(sum(target, []))
    expected_rmse = sqrt(mean_squared_error(tmp_pred, target))
    mae = mean_absolute_error(tmp_pred, target)
    return expected_rmse, mae

In [124]:
 # model
    
lr = 0.01
graphrec = GraphRec(enc_u, enc_v_history, r2e).to(device)
optimizer = torch.optim.RMSprop(graphrec.parameters(), lr=lr, alpha=0.9)

In [125]:
best_rmse = 9999.0
best_mae = 9999.0
endure_count = 0

In [32]:
epochs = 3

for epoch in range(1, epochs + 1):

    train(graphrec, device, train_loader, optimizer, epoch, best_rmse, best_mae)
    expected_rmse, mae = test(graphrec, device, test_loader)
    # please add the validation set to tune the hyper-parameters based on your datasets.

    # early stopping (no validation set in toy dataset)
    if best_rmse > expected_rmse:
        best_rmse = expected_rmse
        best_mae = mae
        endure_count = 0
    else:
        endure_count += 1
    print("rmse: %.4f, mae:%.4f " % (expected_rmse, mae))

    if endure_count > 5:
        break

[1,     0] loss: 0.096, The best rmse/mae: 9999.000000 / 9999.000000
[1,   100] loss: 2.047, The best rmse/mae: 9999.000000 / 9999.000000
rmse: 1.5227, mae:1.3479 
[2,     0] loss: 0.013, The best rmse/mae: 1.522698 / 1.347895
[2,   100] loss: 1.032, The best rmse/mae: 1.522698 / 1.347895
rmse: 0.9026, mae:0.7445 
[3,     0] loss: 0.010, The best rmse/mae: 0.902622 / 0.744537
[3,   100] loss: 0.819, The best rmse/mae: 0.902622 / 0.744537
rmse: 1.2216, mae:1.0759 


### 처음 시작하는 부분 노드 128개 시작 

In [126]:
graphrec.train()
running_loss = 0.0
for i, data in enumerate(train_loader, 0):
    batch_nodes_u, batch_nodes_v, labels_list = data
    optimizer.zero_grad()
    break
#     loss = model.loss(batch_nodes_u.to(device), batch_nodes_v.to(device), labels_list.to(device))
#     loss.backward(retain_graph=True)
#     optimizer.step()
#     running_loss += loss.item()
#     if i % 100 == 0:
#         print('[%d, %5d] loss: %.3f, The best rmse/mae: %.6f / %.6f' % (
#             epoch, i, running_loss / 100, best_rmse, best_mae))
#         running_loss = 0.0

In [127]:
batch_nodes_u, batch_nodes_v, labels_list

(tensor([280, 211, 615, 343, 479, 673, 124, 280,  99, 366, 504,  78, 267, 412,
         647, 430, 703, 143, 151,  96, 365, 399,  10, 590,  24, 281, 335, 525,
         294, 429, 603, 379, 300, 177, 247, 528, 440, 341, 193, 286, 245,  51,
         669, 386, 204, 103, 265, 300, 253, 464, 524, 356, 646, 458,  58, 200,
         298,  78, 669, 424, 638, 626, 126, 341, 694, 699, 169,  44,  36, 247,
          53, 486, 293, 517, 693, 435,  54,  75, 277,  81, 172, 197, 256, 420,
         242, 513, 549,  51,  38, 193, 171, 172, 611, 479,  43, 479, 628, 226,
         625, 227, 430, 568, 172, 315, 422,  49, 468, 683, 669,  74, 680, 468,
          13, 156, 637, 484,  28, 243, 113, 360,  68, 292,  68,  75,  74, 259,
         190,  37]),
 tensor([  32,   47,   47,   37,   64,   76,   58,   24,   48,   88,   61,  607,
          114,   52,  718,   34,   51,  211, 1847,   74,   51,   11,   24,   80,
          255,    8,   78,    8,  122,   13,    6,  102,   46, 1081,   74,   47,
           45, 1255,  727

### UV_Encoder 들어가는 부분 

In [128]:
node = 128

In [ ]:
"""
graphrec.train()
running_loss = 0.0

# 128 batch 만큼 (노드 128개 들어감)
for i, data in enumerate(train_loader, 0):
    batch_nodes_u, batch_nodes_v, labels_list = data
    optimizer.zero_grad()
    break 
    


tmp_history_uv = []
tmp_history_r = []
for node in batch_nodes_u:
    tmp_history_uv.append(history_u_lists[int(node)])
    tmp_history_r.append(history_ur_lists[int(node)])
# neigh_feats = self.aggregator.forward(nodes, tmp_history_uv, tmp_history_r)  # user-item network 
# 위에 부분을 사용한 것. 
neigh_feats = agg_u_history.forward(batch_nodes_u, tmp_history_uv, tmp_history_r)

# class UV_Encoder(nn.Module):
#   def forward(self, nodes):
# forward 부분 

embed_dim = 128
linear1 = nn.Linear(2 * embed_dim, embed_dim) # concat 하니깐 2*embed_dim 하는 거 

self_feats = u2e.weight[batch_nodes_u]
# self-connection could be considered.

# (torch.Tensor, torch.Size([128, 256]))
combined = torch.cat([self_feats, neigh_feats], dim=1)

combined = combined.to('cpu')

#  torch.Size([128, 128]))
combined = F.relu(linear1(combined))

####################################################################################################
class UV_Encoder(nn.Module): 이 부분에서  

aggregation하고서 나온거를 concat해서 128 * 256 사이즈를 다시 렐루와 linear들어가서 128*128로 처리 

enc_u_history == combined 가 됨. 
####################################################################################################

"""

In [129]:
tmp_history_uv = []
tmp_history_r = []
for node in batch_nodes_u:
    tmp_history_uv.append(history_u_lists[int(node)])
    tmp_history_r.append(history_ur_lists[int(node)])

len(tmp_history_uv), len(tmp_history_r)

(128, 128)

In [130]:
len(tmp_history_uv), len(tmp_history_r)

(128, 128)

In [131]:
tmp_len_history_uv = []
tmp_len_history_r = []
for i in range(128):
    tmp_len_history_uv.append(len(tmp_history_uv[i]))
    tmp_len_history_r.append(len(tmp_history_r[i]))

In [132]:
# 각각의 노드가 연결된 수 
print(tmp_len_history_uv, end="")

[33, 30, 5, 38, 42, 35, 17, 33, 12, 37, 39, 94, 22, 10, 16, 19, 37, 31, 79, 16, 39, 13, 21, 36, 37, 13, 12, 29, 10, 15, 15, 16, 37, 33, 11, 43, 13, 55, 133, 15, 14, 34, 110, 14, 25, 4, 20, 37, 15, 40, 89, 28, 34, 29, 48, 39, 37, 94, 110, 38, 11, 37, 42, 55, 35, 34, 21, 22, 41, 11, 211, 25, 7, 8, 23, 33, 29, 76, 31, 81, 63, 26, 52, 35, 34, 4, 39, 34, 161, 133, 34, 63, 18, 42, 102, 42, 102, 10, 49, 33, 19, 34, 63, 47, 11, 36, 14, 16, 110, 146, 38, 14, 62, 9, 81, 29, 20, 3, 26, 34, 6, 64, 6, 76, 146, 48, 70, 64]

In [133]:
# batch수 만큼의 노드에 연결된 rating의 수 (tmp_len_history_uv와 동일한 숫자로 rating이 되어진 것을 확인가능) 즉 관계들의 숫자가 동일하게 구성됨 
print(tmp_len_history_r, end="")
print('\n')

# 한 노드가 연결된 아이템들의 rating 값들 
print(tmp_history_r[:1], end="") # 33개

[33, 30, 5, 38, 42, 35, 17, 33, 12, 37, 39, 94, 22, 10, 16, 19, 37, 31, 79, 16, 39, 13, 21, 36, 37, 13, 12, 29, 10, 15, 15, 16, 37, 33, 11, 43, 13, 55, 133, 15, 14, 34, 110, 14, 25, 4, 20, 37, 15, 40, 89, 28, 34, 29, 48, 39, 37, 94, 110, 38, 11, 37, 42, 55, 35, 34, 21, 22, 41, 11, 211, 25, 7, 8, 23, 33, 29, 76, 31, 81, 63, 26, 52, 35, 34, 4, 39, 34, 161, 133, 34, 63, 18, 42, 102, 42, 102, 10, 49, 33, 19, 34, 63, 47, 11, 36, 14, 16, 110, 146, 38, 14, 62, 9, 81, 29, 20, 3, 26, 34, 6, 64, 6, 76, 146, 48, 70, 64]

[[7, 3, 3, 3, 0, 5, 2, 4, 2, 0, 2, 3, 4, 3, 5, 3, 3, 3, 5, 2, 2, 5, 5, 2, 6, 5, 3, 2, 4, 2, 6, 5, 7]]

### UV_Encoder의 aggreagation(agg_u_history)의 forward 들어가는 부분 

In [134]:
# neigh_feats = self.aggregator.forward(nodes, tmp_history_uv, tmp_history_r)  # user-item network 
# 위에 부분을 사용한 것. 
neigh_feats = agg_u_history.forward(batch_nodes_u, tmp_history_uv, tmp_history_r)

In [135]:
neigh_feats.shape, neigh_feats[0:1]

(torch.Size([128, 128]),
 tensor([[0.0368, 0.2152, 0.0079, 0.0601, 0.0270, 0.1075, 0.0042, 0.1858, 0.0345,
          0.0157, 0.0770, 0.0823, 0.2317, 0.0419, 0.0494, 0.1833, 0.1359, 0.1265,
          0.1890, 0.0112, 0.0667, 0.3008, 0.2181, 0.0790, 0.1828, 0.0664, 0.0842,
          0.1432, 0.0167, 0.0796, 0.0694, 0.0050, 0.0246, 0.0328, 0.0922, 0.1618,
          0.0323, 0.0953, 0.0251, 0.0646, 0.0178, 0.0130, 0.1527, 0.0489, 0.0074,
          0.0468, 0.3014, 0.0379, 0.2098, 0.1054, 0.3498, 0.0567, 0.0335, 0.0480,
          0.0094, 0.1064, 0.0458, 0.1316, 0.2392, 0.1750, 0.0369, 0.1767, 0.0317,
          0.0201, 0.0899, 0.2086, 0.0329, 0.1658, 0.1831, 0.3152, 0.0166, 0.1058,
          0.0984, 0.0012, 0.0588, 0.0386, 0.1685, 0.0260, 0.1205, 0.0406, 0.0243,
          0.0881, 0.0000, 0.1067, 0.1058, 0.2552, 0.1534, 0.1918, 0.1632, 0.0924,
          0.0524, 0.0812, 0.1306, 0.2622, 0.2975, 0.0103, 0.0181, 0.2124, 0.0035,
          0.0923, 0.0144, 0.2624, 0.0423, 0.0780, 0.0732, 0.0455, 0.2509,

In [136]:
"""
# class UV_Encoder(nn.Module):
#   def forward(self, nodes):
# forward 부분 

class UV_Encoder(nn.Module): 이 부분에서  

aggregation하고서 나온거를 concat해서 128 * 256 사이즈를 다시 렐루와 linear들어가서 128*128로 처리 

enc_u_history == combined 가 됨. 
"""


embed_dim = 128
linear1 = nn.Linear(2 * embed_dim, embed_dim)

self_feats = u2e.weight[batch_nodes_u]
# self-connection could be considered.

# (torch.Tensor, torch.Size([128, 256]))
combined = torch.cat([self_feats, neigh_feats], dim=1)

combined = combined.to('cpu')
combined = F.relu(linear1(combined))



In [137]:
combined

tensor([[0.4204, 0.0000, 0.0154,  ..., 0.5163, 0.0000, 0.3760],
        [0.2692, 0.3195, 0.0000,  ..., 0.0123, 0.4432, 0.0000],
        [0.0000, 0.0071, 0.0000,  ..., 0.0000, 0.0000, 0.0427],
        ...,
        [0.0000, 0.1793, 0.0976,  ..., 0.0000, 0.0000, 0.1134],
        [0.0000, 0.0000, 0.0000,  ..., 0.4487, 0.0000, 0.0138],
        [0.1241, 0.7374, 0.0360,  ..., 0.0818, 0.5448, 0.0000]],
       grad_fn=<ReluBackward0>)

### Social_Encoder 클래스 시작 (UV_Encoder 클래스 이후에 combined 값이 enc_u_history 부분임)
###### enc_u_history이게 Social_Encode의 base_model로 들어감

In [138]:
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F

features = lambda nodes: enc_u_history(nodes).t() ## features # lambda nodes: enc_u_history(nodes).t()
social_adj_lists = social_adj_lists ## social_adj_lists # social_adj_lists
aggregator = agg_u_social ## aggregator # agg_u_social
base_model = enc_u_history ##base_model # enc_u_history
embed_dim = embed_dim # 128
device  = device
linear1 = nn.Linear(2 * embed_dim, embed_dim)

In [ ]:
"""
to_neighs = []
for node in batch_nodes_u:
    to_neighs.append(social_adj_lists[int(node)])
neigh_feats = aggregator.forward(batch_nodes_u, to_neighs)  # user-user network
#print(type(self_feats))
self_feats = features(torch.LongTensor(batch_nodes_u.cpu().numpy())).to(device)
#print(type(self_feats))
self_feats = self_feats.t()

# self-connection could be considered.
combined = torch.cat([self_feats, neigh_feats], dim=1)
combined = F.relu(self.linear1(combined))

combined
"""

In [139]:
## social_adj_lists: user's connected neighborhoods

to_neighs = []
for node in batch_nodes_u:
    to_neighs.append(social_adj_lists[int(node)])


In [140]:
to_neighs[:5] # 한 유저가 다른 유저에게 연결된 social 정보를 가지고 있습니다. 여기선 예시로 5개 

[{13, 23, 28, 29, 41, 42, 43, 44, 281, 340, 341},
 {79, 98, 209, 210, 331, 332, 682},
 {411},
 {336},
 {295, 418}]

In [141]:
neigh_feats = aggregator.forward(batch_nodes_u, to_neighs)  # user-user network

In [142]:
# 소셜정보 aggregation 한 결과 
neigh_feats.shape, neigh_feats[0]

(torch.Size([128, 128]),
 tensor([-0.3405, -0.0851, -0.2990, -0.3371, -0.3064, -0.1205,  0.0521,  0.3534,
         -0.2463, -0.3917,  0.1232, -0.3164, -0.0584, -0.3660,  0.0386,  0.1416,
          0.1063,  0.0278,  0.3919,  0.1011, -0.5480,  0.3635, -0.0839, -0.0624,
          0.2265,  0.0551, -0.0936, -0.0104,  0.1786,  0.0962, -0.3777,  0.1163,
         -0.2015,  0.1075, -0.0527, -0.2682, -0.3329,  0.4912,  0.0710,  0.3826,
         -0.5388,  0.0317, -0.2323,  0.1493, -0.1689, -0.2612, -0.2491,  0.0289,
          0.3895, -0.3353,  0.3523,  0.0597, -0.1507, -0.0290,  0.3689, -0.3850,
          0.0025,  0.1479, -0.1980,  0.2675,  0.0015,  0.0585,  0.3016,  0.0732,
          0.2831, -0.1190,  0.0632, -0.0672, -0.3708,  0.3467, -0.3639,  0.5488,
         -0.5733,  0.2925,  0.1091, -0.0438,  0.4186, -0.2221,  0.0546, -0.0355,
         -0.0031, -0.4996, -0.3259, -0.4009, -0.1001, -0.2260,  0.3206, -0.0706,
         -0.0911,  0.0814, -0.0136,  0.3477,  0.2614, -0.3526,  0.0723, -0.0311,
   

In [143]:
#print(type(self_feats))
# features ====> lambda nodes: enc_u_history(nodes).t()
# batch_nodes_u의 전치행렬이 들어감  (enc_u_history(batch_nodes_u).t()[0]와 같음)
self_feats = features(torch.LongTensor(batch_nodes_u.cpu().numpy())).to(device)
self_feats[0], self_feats.shape

(tensor([0.1304, 0.0000, 0.0000, 1.1121, 0.4706, 0.4266, 0.3092, 0.1302, 0.7443,
         0.0000, 0.0054, 0.0000, 0.3650, 0.0000, 0.4820, 0.0000, 0.0000, 0.0000,
         0.0271, 0.0000, 0.8191, 0.0000, 0.2040, 0.2169, 0.0000, 0.3003, 0.0000,
         0.0656, 0.0000, 0.0356, 0.4538, 0.5412, 0.3139, 0.3672, 0.0000, 0.0000,
         0.0000, 0.5696, 0.0000, 0.2573, 0.0000, 0.5056, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.3141, 0.0000, 0.0752, 0.0000, 0.0000, 0.3096, 0.4184,
         0.3706, 0.5858, 0.0031, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5603,
         0.5714, 0.0000, 0.0000, 0.3961, 0.0000, 0.0229, 0.0000, 0.0000, 0.0000,
         0.0000, 0.2193, 0.0000, 0.1474, 0.0000, 0.2324, 0.0468, 0.0000, 0.5453,
         0.0000, 0.0476, 0.5499, 0.4049, 0.0998, 0.2995, 0.5056, 0.1710, 0.0000,
         0.0000, 0.5456, 0.3242, 0.4701, 0.5687, 0.4691, 0.0000, 0.5389, 0.1410,
         0.5128, 0.0000, 0.0000, 0.5462, 0.0000, 0.5890, 0.0000, 0.0000, 0.4968,
         0.0000, 0.2873, 0.0

In [165]:
#print(type(self_feats))
self_feats = self_feats.t()

In [172]:
# self-connection could be considered.
# combined ==> torch.Size([128, 256])
combined = torch.cat([self_feats, neigh_feats], dim=1)
# combined ==> torch.Size([128, 128])
combined = combined.to('cpu')
combined = F.relu(linear1(combined))

enc_u = combined #　enc_u == combined

In [173]:
enc_u

tensor([[0.0700, 0.0000, 0.1852,  ..., 0.1006, 0.0000, 0.1970],
        [0.0000, 0.0243, 0.0157,  ..., 0.0000, 0.3118, 0.3124],
        [0.0000, 0.1815, 0.0000,  ..., 0.1086, 0.2316, 0.0000],
        ...,
        [0.1322, 0.0000, 0.0000,  ..., 0.0090, 0.0000, 0.0000],
        [0.0000, 0.2337, 0.2874,  ..., 0.3724, 0.0000, 0.0000],
        [0.0078, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.1757]],
       grad_fn=<ReluBackward0>)

In [ ]:
class GraphRec(nn.Module):

    def __init__(self, enc_u, enc_v_history, r2e):
        super(GraphRec, self).__init__()
        self.enc_u = enc_u
        self.enc_v_history = enc_v_history
        self.embed_dim = enc_u.embed_dim

        self.w_ur1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_ur2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_uv1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.w_uv2 = nn.Linear(self.embed_dim, 16)
        self.w_uv3 = nn.Linear(16, 1)
        self.r2e = r2e
        self.bn1 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn2 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn3 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn4 = nn.BatchNorm1d(16, momentum=0.5)
        self.criterion = nn.MSELoss()

    def forward(self, nodes_u, nodes_v):
        embeds_u = self.enc_u(nodes_u)
        embeds_v = self.enc_v_history(nodes_v)

        x_u = F.relu(self.bn1(self.w_ur1(embeds_u)))
        x_u = F.dropout(x_u, training=self.training)
        x_u = self.w_ur2(x_u)
        x_v = F.relu(self.bn2(self.w_vr1(embeds_v)))
        x_v = F.dropout(x_v, training=self.training)
        x_v = self.w_vr2(x_v)

        x_uv = torch.cat((x_u, x_v), 1)
        x = F.relu(self.bn3(self.w_uv1(x_uv)))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.bn4(self.w_uv2(x)))
        x = F.dropout(x, training=self.training)
        scores = self.w_uv3(x)
        return scores.squeeze()

    def loss(self, nodes_u, nodes_v, labels_list):
        scores = self.forward(nodes_u, nodes_v)
        return self.criterion(scores, labels_list)


def train(model, device, train_loader, optimizer, epoch, best_rmse, best_mae):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        batch_nodes_u, batch_nodes_v, labels_list = data
        optimizer.zero_grad()
        loss = model.loss(batch_nodes_u.to(device), batch_nodes_v.to(device), labels_list.to(device))
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f, The best rmse/mae: %.6f / %.6f' % (
                epoch, i, running_loss / 100, best_rmse, best_mae))
            running_loss = 0.0
    return 0


def test(model, device, test_loader):
    model.eval()
    tmp_pred = []
    target = []
    with torch.no_grad():
        for test_u, test_v, tmp_target in test_loader:
            test_u, test_v, tmp_target = test_u.to(device), test_v.to(device), tmp_target.to(device)
            val_output = model.forward(test_u, test_v)
            tmp_pred.append(list(val_output.data.cpu().numpy()))
            target.append(list(tmp_target.data.cpu().numpy()))
    tmp_pred = np.array(sum(tmp_pred, []))
    target = np.array(sum(target, []))
    expected_rmse = sqrt(mean_squared_error(tmp_pred, target))
    mae = mean_absolute_error(tmp_pred, target)
    return expected_rmse, mae

In [176]:
enc_u = enc_u
enc_v_history = enc_v_history
embed_dim = embed_dim

w_ur1 = nn.Linear(embed_dim, embed_dim)
w_ur2 = nn.Linear(embed_dim, embed_dim)
w_vr1 = nn.Linear(embed_dim, embed_dim)
w_vr2 = nn.Linear(embed_dim, embed_dim)
w_uv1 = nn.Linear(embed_dim * 2, embed_dim)
w_uv2 = nn.Linear(embed_dim, 16)
w_uv3 = nn.Linear(16, 1)
r2e = labels_list ## r2e # labels_list
bn1 = nn.BatchNorm1d(embed_dim, momentum=0.5)
bn2 = nn.BatchNorm1d(embed_dim, momentum=0.5)
bn3 = nn.BatchNorm1d(embed_dim, momentum=0.5)
bn4 = nn.BatchNorm1d(16, momentum=0.5)
criterion = nn.MSELoss()

In [180]:
#embeds_u = enc_u(nodes_u)
#embeds_v = enc_v_history(nodes_v)

x_u = F.relu(bn1(w_ur1(enc_u)))

In [181]:
x_u.shape

torch.Size([128, 128])

In [182]:
x_u = w_ur2(x_u)
x_u.shape

torch.Size([128, 128])

In [190]:
x_v = F.relu(bn2(w_vr1(enc_v_history(batch_nodes_v).cpu())))
x_v.shape

torch.Size([128, 128])

In [191]:
x_v = w_vr2(x_v)

In [198]:
x_uv = torch.cat((x_u, x_v), 1)
x_uv.shape

torch.Size([128, 256])

In [199]:
x = F.relu(bn3(w_uv1(x_uv)))
x.shape

torch.Size([128, 128])

In [200]:
x = F.relu(bn4(w_uv2(x)))
x.shape

torch.Size([128, 16])

In [203]:
scores = w_uv3(x)
print(scores.shape)

scores.squeeze()
print(scores.shape, "squeeze()")


torch.Size([128, 1])
torch.Size([128, 1]) squeeze()


In [208]:
scores

tensor([[ 0.0702],
        [-0.0099],
        [ 0.1754],
        [-0.0233],
        [ 0.1246],
        [-0.0512],
        [-0.5902],
        [ 0.3616],
        [ 0.2391],
        [ 0.6653],
        [ 0.1989],
        [ 0.5786],
        [-0.0733],
        [ 0.6730],
        [-0.0751],
        [ 0.0692],
        [-0.2070],
        [ 0.0930],
        [ 0.3057],
        [-0.1126],
        [ 0.1526],
        [ 0.1043],
        [-0.1905],
        [ 0.2896],
        [ 0.0617],
        [-0.0708],
        [-0.0182],
        [ 0.5053],
        [-0.0786],
        [-0.3031],
        [ 0.6304],
        [-0.1517],
        [ 0.6497],
        [ 1.0408],
        [ 0.0346],
        [ 0.0533],
        [-0.2603],
        [ 0.1876],
        [-0.2263],
        [-0.6179],
        [-0.1443],
        [-0.0993],
        [ 0.4771],
        [ 0.8111],
        [ 0.2421],
        [ 0.0863],
        [ 0.8681],
        [ 1.1338],
        [ 0.5297],
        [ 0.2577],
        [ 0.0065],
        [ 0.1068],
        [-0.

In [210]:
loss_func = nn.MSELoss()
loss = loss_func(scores, labels_list)

/home/user/anaconda3/envs/graph-embedding/lib/python3.8/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [211]:
loss

tensor(9.5524, grad_fn=<MseLossBackward>)